<a href="https://colab.research.google.com/github/zikili/Python-AI-Algorithms/blob/main/Matala1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
from queue import Queue
from queue import PriorityQueue
from collections import defaultdict
from collections import deque
import sys



def findPosition(state, num):
    numPos = [0, 0]
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] == num:
                numPos[0] = i
                numPos[1] = j
                return numPos
    return numPos

def solveIDAstar(initialState):
    maxDepth = h_calculateManhattanDistance(initialState)
    while maxDepth < sys.maxsize:
        depth, lst = solveIDAstarhelp(initialState, dict(), set(), deque(), maxDepth, 0, sys.maxsize)

        if depth == -1:
            return lst
        if depth > maxDepth:
            maxDepth = depth

def solveIDAstarhelp(currentState, heruistics, closed, path, maxDepth, currentDepth, minimum):
    queue = deque()
    key = getKey(currentState)
    closed.add(key)
    lst = None
    heruistics[key] = f_calculateManhattanDistance(currentState, currentDepth)
    path.append(currentState)
    if currentState == goalState:
        lst = list(path)
        return -1, lst

    successors = getSuccessors(currentState)
    for successor in successors:
        successorKey = getKey(successor)
        successorDist = f_calculateManhattanDistance(successor, currentDepth + 1)
        if successorKey not in closed or successorDist < heruistics.get(successorKey, successorDist):
            if successorDist <= maxDepth:
                if successorDist <= heruistics.get(successorKey, successorDist):
                    heruistics[successorKey] = successorDist
                    queue.append(successor)
            else:
                minimum = min(minimum, successorDist)

    while queue:
        state = queue.popleft()
        minimum, lst = solveIDAstarhelp(state, heruistics, closed, path, maxDepth, currentDepth + 1, minimum)
        if(minimum==-1):
          return minimum, lst

    path.pop()
    return minimum, lst

def solveIDS(initialState):
    maxDepth = 0
    while maxDepth <sys.maxsize:
        check = solveDFS(initialState, dict(), set(), deque(), maxDepth, 0)
        if check is not None:
            return check;
        maxDepth += 1;

def solveDFS(currentState, depth, closed, path, maxDepth, currentDepth):
    key = getKey(currentState)
    closed.add(key)
    depth[key] = currentDepth
    path.append(currentState)
    if currentState == goalState:
        solution = list(path)
        return solution
    elif maxDepth == currentDepth:
        path.pop()
        return None

    successors = getSuccessors(currentState)
    for successor in successors:
        successorKey = getKey(successor)
        if successorKey not in closed or currentDepth < depth[successorKey]:
            solution = solveDFS(successor, depth, closed, path, maxDepth, currentDepth + 1)
            if solution is not None:
                return solution

    path.pop()
    return None

def solveBFS(initialState):
    queue = Queue()
    closed = set()
    direction = {}
    parent = {}

    queue.put(initialState)
    while not queue.empty():
        currentState = queue.get()
        closed.add(getKey(currentState))

        if currentState == goalState:
            printSolution(parent, direction, currentState)
            return

        successors = getSuccessors(currentState)
        for successor in successors:
            successorKey = getKey(successor)
            if successorKey not in closed and successorKey not in parent:
                queue.put(successor)
                parent[successorKey] = currentState
                direction[successorKey] = getMoveDirection(currentState, successor)

    print("No solution found!")

def solveAstar(initialState):
    depth = {}
    queue = PriorityQueue()
    closed = set()
    direction = {}
    parent = {}
    insertion_order = 0  #counter to track insertion order

    initialKey = getKey(initialState)
    depth[initialKey] = 0
    queue.put((f_calculateManhattanDistance(initialState, depth[initialKey]), insertion_order, initialState))

    while not queue.empty():
        _, _, currentState = queue.get()  #extracting priority and insertion order along with currentState
        currentKey = getKey(currentState)
        closed.add(currentKey)

        if currentState == goalState:
            printSolution(parent, direction, currentState)
            return

        successors = getSuccessors(currentState)
        for successor in successors:
            successorKey = getKey(successor)
            new_depth = depth[currentKey] + 1

            if f_calculateManhattanDistance(currentState, new_depth) < f_calculateManhattanDistance(successor, depth.get(successorKey, 0)) or (successorKey not in closed and successorKey not in parent):
                depth[successorKey] = new_depth
                queue.put((f_calculateManhattanDistance(successor, depth[successorKey]), insertion_order, successor))
                parent[successorKey] = currentState
                direction[successorKey] = getMoveDirection(currentState, successor)
                insertion_order += 1

    print("No solution found!")

def getSuccessors(currentState):
    successors = []
    dx = [1, -1, 0, 0]
    dy = [0, 0, 1, -1]
    zeroPos = findZeroPosition(currentState)
    i = zeroPos[0]
    j = zeroPos[1]
    for k in range(4):
        ni = i + dx[k]
        nj = j + dy[k]
        if 0 <= ni < len(currentState) and 0 <= nj < len(currentState[0]):
            nextState = cloneState(currentState)
            nextState[i][j] = nextState[ni][nj]
            nextState[ni][nj] = 0
            successors.append(nextState)
    return successors

def getMoveDirection(currentState, nextState):
    dx = [1, -1, 0, 0]
    dy = [0, 0, 1, -1]
    dir = ["U", "D", "L", "R"]
    for k in range(4):
        tempState = cloneState(currentState)
        zeroPos = findZeroPosition(tempState)
        ni = zeroPos[0] + dx[k]
        nj = zeroPos[1] + dy[k]
        if 0 <= ni < len(currentState) and 0 <= nj < len(currentState[0]) and nextState[ni][nj] == 0:
            return dir[k]
    return ""

def findZeroPosition(state):
    numPos = [0, 0]
    for i in range(len(state)):
        for j in range(len(state[0])):
            if state[i][j] == 0:
                numPos[0] = i
                numPos[1] = j
                return numPos
    return numPos

def getKey(state):
    return ''.join(str(cell) for row in state for cell in row)

def cloneState(state):
    return [row.copy() for row in state]

def printSolution(parent, direction, currentState):
    solution = []
    solutionMat = []
    string=""
    key = getKey(currentState)
    while key in parent:
        solution.insert(0, direction[key])
        solutionMat.insert(0, currentState)
        currentState = parent[key]
        key = getKey(currentState)
    solutionMat.insert(0, currentState)
    for move in solution:
        string+=move
    with open('output.txt', 'w') as f:
      f.write(string)

# def printState(state):
#     for row in state:
#         for cell in row:
#             print(cell, end=" ")
#         print()

# def print_solution(solution):
#     import sys
#     moves = []

#     if solution and solution:
#         next_state = solution[0]
#         for i in range(len(solution) - 1):
#             state = solution[i]
#             next_state = solution[i + 1]
#             moves.append(getMoveDirection(state, next_state))
#             printState(state)
#             print()

#         printState(next_state)
#         print()
#         print("Solution directions:")
#         for move in moves:
#             print(move, end=" ")

#         print()
#         print("Total moves:", len(solution) - 1)
#     else:
#         print("No solution found!", file=sys.stderr)


def print_sol(solution):
    import sys
    moves=[]
    sol=""
    if solution and solution:
        next_state = solution[0]
        for i in range(len(solution) - 1):
            state = solution[i]
            next_state = solution[i + 1]
            moves.append(getMoveDirection(state, next_state))
    for move in moves:
          sol+=move
    return sol

def generateGoalState(n):
    goal = [[0] * n for _ in range(n)]
    num = 1
    for i in range(n):
        for j in range(n):
            goal[i][j] = num
            num += 1
    goal[n - 1][n - 1] = 0
    return goal

# def generateRandomState(n):
#     numbers = list(range(n * n))
#     random.shuffle(numbers)

#     state = [[0] * n for _ in range(n)]
#     index = 0
#     for i in range(n):
#         for j in range(n):
#             state[i][j] = numbers[index]
#             index += 1
#    return state

def f_calculateManhattanDistance(currentState, depth):
    return h_calculateManhattanDistance(currentState) + depth

def h_calculateManhattanDistance(currentState):
    dist = 0
    for i in range(n):
        for j in range(n):
            goalPosition = findPosition(goalState, currentState[i][j])
            dist += h_calculateManhattanDistanceHelp(i, j, goalPosition[0], goalPosition[1])
    return dist

def h_calculateManhattanDistanceHelp(point1_i, point1_j, point2_i, point2_j):
    return abs(point1_i - point2_i) + abs(point1_j - point2_j)

def stringToMat(string, n):
    elements = string.split("-")
    matrix = [elements[i:i+n] for i in range(0, len(elements), n)]
    return matrix


with open('input.txt', 'r') as file:
    counter=0
    algNum=0
    n=0
    mat=[]
    for line in file:
      if(counter==0):
        algNum=int(line.strip())
      elif(counter==1):
        n=int(line.strip())
      else:
        mat=line.strip()
      counter+=1

def stringToMat(string, n):
    elements = string.split("-")
    matrix = [[int(element) for element in elements[i:i+n]] for i in range(0, len(elements), n)]
    return matrix


initialState=stringToMat(mat,n)
goalState = generateGoalState(n)

if(algNum==1):
  idsSolution = None
  idsSolution=solveIDS(initialState)
  string=(print_sol(idsSolution))
  print(string)
  with open('output.txt', 'w') as f:
    f.write(string)

elif(algNum==2):
  solveBFS(initialState)

elif(algNum==3):
  solveAstar(initialState)

elif(algNum==4):
  idaStarSolution = None
  idaStarSolution=solveIDAstar(initialState)
  string=(print_sol(idaStarSolution))
  print(string)
  with open('output.txt', 'w') as f:
    f.write(string)

else:
  dfsSolution = None
  dfsSolution = solveDFS(initialState, dict(), set(), deque(), n * n, 0)
  string=(print_sol(dfsSolution))
  print(string)
  with open('output.txt', 'w') as f:
    f.write(string)













